In [1]:
import pandas as pd
from konlpy.tag import Komoran, Kkma, Okt
import re
import numpy as np
from tqdm import tqdm

tqdm.pandas()

In [2]:
data = pd.read_csv("../database/review_pseudolabeled.csv", encoding="utf-8")
data[:5]

,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user,sentence,fixed,label,probability
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028,영등포에서는 이곳만 와요 너무 좋습니다,영등포에서는 이곳만 와요,1,97.836401
1,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028,영등포에서는 이곳만 와요 너무 좋습니다,너무 좋습니다,1,99.999919
2,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,체크인 시간이 너무너무 깁니다,0,91.076837
3,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,빨리할 수 있는 방법을 구상해야 할 것 같습니다,0,56.304458
4,609182b3f5c0adc042e03b0a,5,잘 머물다 갑니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12752105,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,5,그때그시절처렁,잘 머물다 갑니다,잘 머물다 갑니다,1,99.999904


In [3]:
with open("../vocab/kor_stop.txt", "r", encoding="utf-8") as f:
    list_file = f.readlines()
stop_words = [line.rstrip("\n") for line in tqdm(list_file)]
stop_words

100%|██████████| 214/214 [00:00<00:00, 1146335.96it/s]


['을',
 '를',
 '그',
 '은',
 '는',
 '의',
 '하다',
 '이',
 '가',
 '에',
 '에서',
 '거',
 '으로',
 '요',
 '로',
 '다만',
 '더',
 '인',
 '때',
 '건',
 '점',
 '나',
 '되어다',
 '그렇다',
 '서',
 '면',
 '이라',
 '두',
 '제',
 '정말',
 '하',
 '에요',
 '라',
 '내',
 '번',
 '이렇다',
 '라고',
 '네',
 '자',
 '옴',
 '재',
 '랑',
 '또',
 '음',
 '에서는',
 '이고',
 '어',
 '여기',
 '저기',
 '그냥',
 '와',
 '부터',
 '까지',
 '불다',
 '추다',
 '흠',
 '상',
 '꽝',
 '여',
 '측',
 '지금',
 '심지어',
 '잘',
 '터',
 '처럼',
 '땜',
 '흐',
 '거니',
 '건가',
 '그건',
 '오늘',
 '이전',
 '뇨',
 '이미',
 '에만',
 '흑',
 '로는',
 '니까',
 '단지',
 '무슨',
 '에서도',
 '걸',
 '점점',
 '여서',
 '간',
 '딱',
 '그게',
 '에게는',
 '아예',
 '움',
 '옴',
 '이네',
 '라고',
 '이렇다',
 '근데',
 '서',
 '면',
 '도',
 '있',
 '았',
 '었',
 '고',
 '어요',
 '시',
 '안',
 'ㄴ',
 '되',
 '좋',
 '네요',
 '없',
 '어서',
 '좀',
 '습니다',
 '아요',
 'ㄹ',
 '게',
 'ㅁ',
 '주',
 '너무',
 '기',
 '지',
 '아',
 '아쉽',
 'ㅂ니다',
 '조금',
 '지만',
 '같',
 '아서',
 '것',
 '다',
 '않',
 '많',
 '는데',
 '오',
 '방',
 '나오',
 '빼',
 '아니',
 '쓰',
 '보이',
 '괜찮',
 '듯',
 '분',
 '진짜',
 '끌',
 '밑',
 '일',
 '많이',
 '개',
 '이번',
 '힘들',
 '남',
 '그래도',
 '바로',

In [4]:
hotel_name = list(data.groupby("hotelName").count().index)
print(hotel_name)

groups = data.groupby(data.hotelName)

['JK 블라썸 호텔', '골드리버 호텔', '글래드 강남 코엑스센터', '글래드 라이브 강남', '글래드 마포', '글래드 여의도', '나인트리 프리미어 명동2', '나인트리 호텔 동대문', '노보텔 앰배서더 서울 용산', '더 스테이트 선유', '도미인 서울 강남', '라마다 서울 강남', '롯데시티호텔 구로', '롯데시티호텔 명동', '머큐어 앰배서더 강남 쏘도베', '메리골드 호텔', '신라스테이 광화문', '신라스테이 구로', '신라스테이 마포', '신라스테이 서대문', '신라스테이 서초', '신라스테이 역삼', '아만티 호텔 서울 홍대', '어반플레이스 강남', '여의도 M 호텔', '영등포 GMS 호텔', '오라카이 대학로 호텔', '이비스 스타일 앰배서더 서울 용산', '인터시티 서울 호텔', '잠실2.4호텔', '페어필드 바이 메리어트 서울', '포포인츠 바이 쉐라톤 서울 구로', '호텔 더 디자이너스 여의도', '호텔 스카이파크 킹스타운 동대문점', '호텔 크레센도 서울', '호텔 피제이 명동', '호텔해담채가산', '홀리데이인 익스프레스 서울 홍대']


In [12]:
for i, name in tqdm(enumerate(hotel_name)):
    globals()["df_{}".format(i)] = groups.get_group(name)

df_0[:5]

38it [00:00, 1569.52it/s]


,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user,sentence,fixed,label,probability
114154,60918512f5c0adc042e15ba2,5,추천합니다,5,2021-05-04,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,5,5,12754784,슈페리어 더블(리버뷰),5,꿀떡꿀떡나그네0033,추천합니다,추천합니다,1,99.320898
114155,60918512f5c0adc042e15bbd,4,연박으로 이용했는데 개인 물건을 마음대로 치우고 버리셨나 봐요. 중요한 물건은 아니...,5,2021-04-13,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,2,5,12520516,슈페리어 더블(리버뷰),3,불타는골목길0002,연박으로 이용했는데 개인 물건을 마음대로 치우고 버리셨나 봐요. 중요한 물건은 아니...,연박으로 이용했는데 개인 물건을 마음대로 치우고 버리셨나 봐요 .,1,80.454467
114156,60918512f5c0adc042e15bbd,4,연박으로 이용했는데 개인 물건을 마음대로 치우고 버리셨나 봐요. 중요한 물건은 아니...,5,2021-04-13,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,2,5,12520516,슈페리어 더블(리버뷰),3,불타는골목길0002,연박으로 이용했는데 개인 물건을 마음대로 치우고 버리셨나 봐요. 중요한 물건은 아니...,중요한 물건은 아니었지만 없어진 것이 있어서 매우 황당했습니다 .,1,73.011482
114157,60918512f5c0adc042e15baf,5,이주 아주 좋았어요 위에 테라스도 괜찮고 분위기 짱짱,5,2021-04-22,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,5,5,12620124,[Fall in the Blossom PKG] 슈페리어 더블 + 3만원 레스토랑이용권,5,아기염소온리유0042,이주 아주 좋았어요 위에 테라스도 괜찮고 분위기 짱짱,이주 아주 좋았어요,1,99.999894
114158,60918512f5c0adc042e15baf,5,이주 아주 좋았어요 위에 테라스도 괜찮고 분위기 짱짱,5,2021-04-22,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,5,5,12620124,[Fall in the Blossom PKG] 슈페리어 더블 + 3만원 레스토랑이용권,5,아기염소온리유0042,이주 아주 좋았어요 위에 테라스도 괜찮고 분위기 짱짱,위에 테라스도 괜찮고 분위기 짱짱,1,99.999121


In [13]:
df_0_neg = df_0[df_0["label"]==0].copy()

In [28]:
df_0_neg["sentence"] = df_0_neg["sentence"].progress_apply(lambda x: re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))
df_0_neg["sentence"]

100%|██████████| 826/826 [00:00<00:00, 486313.18it/s]


114160                           주차장 협소
114165                      가운 암 커튼 가성비
114173                        만족 주차장 협소
114174                          서비스 형편없
114175                          서비스 형편없
                      ...              
117156        편의점 저녁 간단히 구입 편하 아침 거리 소리
117159                     신축 깨끗 접근성 조용
117167       청결 방음 약간 문제 여자 직원 친절 객실 야경
117168    지난번 숙박 한강 어제 완전 아파트 화장실 비치 비누
117169    지난번 숙박 한강 어제 완전 아파트 화장실 비치 비누
Name: sentence, Length: 826, dtype: object

In [29]:
document_count = len(df_0_neg)
document_count

826

In [16]:
komoran = Komoran(userdic="../vocab/user_dic.txt")
kkma = Kkma()
okt = Okt()

In [24]:
def tokenize(text):
    pos_tag = ["NNG", "NNP", "NNB", "NP", "NR", "VA", "MM", "MAG", "XR"]
    tokens_pos = komoran.pos(text)
    tokens_word = []

    for tag in tokens_pos:
        if tag[1] in pos_tag:
            if tag[0] not in stop_words:
                tokens_word.append(tag[0])

    return re.sub("\.",""," ".join(tokens_word))

In [31]:
df_0_neg["sentence"] = df_0_neg["sentence"].progress_apply(lambda x: tokenize(x))
df_0_neg["sentence"][:5]

100%|██████████| 826/826 [00:00<00:00, 2256.77it/s]


114160         주차장 협소
114165    가운 암 커튼 가성비
114173      만족 주차장 협소
114174         서비스 형편
114175         서비스 형편
Name: sentence, dtype: object

In [35]:
documents = df_0_neg["sentence"].to_list()
documents[:5]

['주차장 협소', '가운 암 커튼 가성비', '만족 주차장 협소', '서비스 형편', '서비스 형편']

In [44]:
def get_term_frequency(document, word_dict=None):
    if word_dict is None:
        word_dict = {}
    words = document.split()

    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])

    return pd.Series(word_dict, dtype="float64").sort_values(ascending=False)

In [45]:
def get_document_frequency(documents):
    dicts = []
    vocab = set([])
    df = {}

    for d in documents:
        tf = get_term_frequency(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())

    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1

    return pd.Series(df, name="df", dtype="float64").sort_values(ascending=False)

In [46]:
def get_tf(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]

    stats = []
    for word, freq in vocab.items():
        tf_v = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tf_v += [tfs[idx][word]]
            else:
                tf_v += [0]
        stats.append((word, freq, *tf_v))

    column_name = ["word", "totalFrequency"]
    for i in range(1, len(docs) + 1):
        column_name.append("document" + str(i))

    return pd.DataFrame(stats, columns=column_name).sort_values("totalFrequency", ascending=False)

In [47]:
tf_matrix = get_tf(documents)
df_matrix = get_document_frequency(documents)

In [48]:
df_matrix

침대     111.0
객실     103.0
깔끔     100.0
불편      99.0
먼지      96.0
       ...  
핏자국      1.0
지원       1.0
인근       1.0
첨        1.0
쿠션       1.0
Name: df, Length: 1302, dtype: float64

In [49]:
tf_matrix

,word,totalFrequency,document1,document2,document3,document4,document5,document6,document7,document8,...,document817,document818,document819,document820,document821,document822,document823,document824,document825,document826
0,시설,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,직원,155.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,침대,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,객실,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,편의점,121.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,공항,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1169,첫,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1170,방기,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1171,고정,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
def get_ntf(matrix):
    max_btf = max(matrix["totalFrequency"])
    total_btf = sum(matrix["totalFrequency"])
    col_names = list(matrix.columns)[2:]
    matrix["ntf1"] = matrix["totalFrequency"].progress_apply(lambda x: x / max_btf)
    matrix_ntf2 = matrix[col_names].copy()
    matrix_ntf2 = matrix_ntf2.progress_apply(lambda x: x / total_btf, axis=1)
    matrix_ntf2["ntf2"] = matrix_ntf2.progress_apply(sum, axis=1)
    matrix["ntf2"] = matrix_ntf2["ntf2"]

    return matrix[["word", "totalFrequency", "ntf1", "ntf2"]]

In [51]:
ntf_matrix = get_ntf(tf_matrix)

100%|██████████| 1302/1302 [00:00<00:00, 17656.69it/s]


In [52]:
def get_ntf_idf(ntf, df):
    ntf = ntf.set_index("word")
    ntf_idf = pd.concat([ntf, df], axis=1)

    def get_btfidf(scores):
        global document_count
        return (np.log(scores["totalFrequency"]) + 1.0) * np.log(document_count / scores["df"])

    def get_ntf1idf(scores):
        global document_count
        return (np.log(scores["ntf1"]) + 1.0) * np.log(document_count / scores["df"])

    def get_ntf2idf(scores):
        global document_count
        return (np.log(scores["ntf2"]) + 1.0) * np.log(document_count / scores["df"])

    ntf_idf["btf_idf"] = ntf_idf.progress_apply(get_btfidf, axis=1)
    ntf_idf["ntf1_idf"] = ntf_idf.progress_apply(get_ntf1idf, axis=1)
    ntf_idf["ntf2_idf"] = ntf_idf.progress_apply(get_ntf2idf, axis=1)

    btf_rank = ntf_idf["btf_idf"].sort_values(ascending=False)
    ntf1_rank = ntf_idf["ntf1_idf"].sort_values(ascending=False)
    ntf2_rank = ntf_idf["ntf2_idf"].sort_values(ascending=False)

    return btf_rank, ntf1_rank, ntf2_rank

In [53]:
btf_rank , ntf1_rank, ntf2_rank = get_ntf_idf(ntf_matrix, df_matrix)

100%|██████████| 1302/1302 [00:00<00:00, 67146.00it/s]


In [ ]:
def calculate_save():
    for h_name in tqdm(hotel_name):
        docs = data[data["hotelName"] == h_name]["fixed"].to_list()
        document_count = len(docs)
        tf_matrix = get_tf(docs)
        df_matrix = get_document_frequency(docs)
        ntf_matrix = get_ntf(tf_matrix)
        # btf_rank , ntf1_rank, ntf2_rank = get_ntf_idf(ntf_matrix, df_matrix, document_count)
        ntf1_rank = get_ntf_idf(ntf_matrix, df_matrix, document_count)

        # btf_rank.to_csv('../../Data/keywords/btf_'+h_name+'.csv', encoding='utf-8-sig')

        ntf1_rank.to_csv(
            "../../Data/keywords/ntf1_" + h_name + ".csv", encoding="utf-8-sig"
        )

        # ntf2_rank.to_csv('../../Data/keywords/ntf2_'+h_name+'.csv', encoding='utf-8-sig')